In [4]:
%%capture
!pip install torchinfo
!pip install LTNtorch
!pip install timm
!pip install ray

In [ ]:
!pip install hyperopt

In [1]:
# Standard Libraries
import os
from typing import Tuple, List, Dict
from time import time
from datetime import datetime
from pathlib import Path
import sys

# Data Manipulation and Visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, transform
from tqdm import tqdm
import csv

# Machine Learning Frameworks
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms, utils
from torchsummary import summary
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
    f1_score,
)

# Computer Vision
import cv2

# Model Architectures
import timm

# Experimentation and Optimization
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, AsyncHyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from hyperopt import hp

# Logic Tensor Network
import ltn

# Miscellaneous
import requests
import re

# Parallel Processing
from typing import List
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch

# Other
from torchvision import datasets
from sklearn.metrics import ConfusionMatrixDisplay
from abc import ABC
import glob
import pickle

In [2]:
def create_label_dict(category_dict):
    # Initialize dictionaries
    coarse_label_dict = {}
    fine_label_dict = {}
    coarse_to_fine = {}

    # Assign numerical labels
    coarse_label_counter = 0
    fine_label_counter = len(category_dict)

    # Iterate through the input dictionary
    for category, labels in category_dict.items():
        # Assign a numerical label to the coarse category
        coarse_label_dict[category] = coarse_label_counter

        # Create an empty list to store fine labels for this coarse category
        coarse_to_fine[coarse_label_counter] = []

        # Iterate through labels in the category
        for label in labels:
            # Assign a numerical label to the fine label
            fine_label_dict[label] = fine_label_counter

            # Add the fine label to the list of fine labels for this coarse category
            coarse_to_fine[coarse_label_counter].append(fine_label_counter)

            # Increment the fine label counter
            fine_label_counter += 1

        # Increment the coarse label counter
        coarse_label_counter += 1

    # Return the resulting dictionaries
    return coarse_label_dict, fine_label_dict, coarse_to_fine


def create_one_hot_tensors(fine_label_dict, coarse_label_dict):
    l = {}
    num_labels = len(coarse_label_dict)+len(fine_label_dict)
    for label in range(num_labels):
        one_hot = torch.zeros(num_labels)
        one_hot[label] = 1.0
        l[label] = ltn.Constant(one_hot, trainable=True)
    return l


def create_inverse_dict(coarse_label_dict, fine_label_dict):
    inverse_dict = {}
    for label, value in coarse_label_dict.items():
        inverse_dict[value] = label

    for label, value in fine_label_dict.items():
        inverse_dict[value] = label

    return inverse_dict


def extract_labels(folder_path):
    parts = folder_path.split(os.path.sep)
    coarse_label = parts[-2]
    fine_label = parts[-1]
    return coarse_label, fine_label


def search_for_images_and_labels(folder):
    data = []
    for image_path in glob.glob(os.path.join(folder, "*.jpg")):
        coarse_label, fine_label = extract_labels(folder)
        data.append({
            'completed_relative_path': os.path.abspath(image_path),
            'Coarse label': coarse_label,
            'fine label': fine_label
        })
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            data.extend(search_for_images_and_labels(subfolder_path))
    return data


def process_image_folders(base_train_folder, base_test_folder):
    # Process train folder
    train_data = search_for_images_and_labels(base_train_folder)
    df_train = pd.DataFrame(train_data)
    df_train['Coarse label'] = df_train['Coarse label'].replace(
        coarse_label_dict)
    df_train['fine label'] = df_train['fine label'].replace(fine_label_dict)

    # Filter train dataset
    coarse_train_labels = [label for _, label in coarse_label_dict.items()]
    fine_train_labels = [label for _, label in fine_label_dict.items()]
    filter_train_coarse = df_train['Coarse label'].isin(coarse_train_labels)
    filter_train_fine = df_train['fine label'].isin(fine_train_labels)
    df_train = df_train[filter_train_coarse &
                        filter_train_fine].reset_index(drop=True)

    # Process test folder
    test_data = search_for_images_and_labels(base_test_folder)
    df_test = pd.DataFrame(test_data)
    df_test['Coarse label'] = df_test['Coarse label'].replace(
        coarse_label_dict)
    df_test['fine label'] = df_test['fine label'].replace(fine_label_dict)

    # Filter test dataset
    coarse_test_labels = [label for _, label in coarse_label_dict.items()]
    fine_test_labels = [label for _, label in fine_label_dict.items()]
    filter_test_coarse = df_test['Coarse label'].isin(coarse_test_labels)
    filter_test_fine = df_test['fine label'].isin(fine_test_labels)
    df_test = df_test[filter_test_coarse &
                      filter_test_fine].reset_index(drop=True)

    return df_train, df_test


class DatasetGenerator():
    """
    Create a dataloader to efficiently get data. The argument include:
        - dataset: the dataframe containing image_path and label
        - image_resize: size of the image
    """

    def __init__(self, dataset, image_resize):
        self.dataset = dataset
        self.image_resize = image_resize

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        # Get index
        idx = index % len(self.dataset)
        image_path = self.dataset['completed_relative_path'][idx]
        image = Image.open(image_path)
        image_rgb = Image.new("RGB", image.size)
        image_rgb.paste(image)

        coarse_label = self.dataset['Coarse label'][idx]
        fine_label = self.dataset['fine label'][idx]

        # Change image to float, resize image and

        imagenet_stats = ([0.5] * 3, [0.5] * 3)
        preprocess = transforms.Compose([
            transforms.Resize((self.image_resize, self.image_resize)),
            transforms.RandomResizedCrop(
                max((self.image_resize, self.image_resize))),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(*imagenet_stats)
        ])

        image_rgb = preprocess(image_rgb)

        return image_rgb, coarse_label, fine_label, image_path


def create_data_loaders(df_train, df_test, image_resize, batch_size, num_coarse_label, num_all_label):
    """
    Create data loaders for the training and testing datasets.

    Args:
        df_train (pd.DataFrame): Training dataset.
        df_test (pd.DataFrame): Testing dataset.
        image_resize (int): Size to which images will be resized.
        batch_size (int): Number of samples in each batch.
        num_coarse_label (int): Number of coarse labels.
        num_all_label (int): Total number of labels including fine and coarse labels.

    Returns:
        DataLoader: Training data loader.
        DataLoader: Testing data loader.
    """
    train_dataset = DatasetGenerator(df_train, image_resize)
    test_dataset = DatasetGenerator(df_test, image_resize)

    # Compute class weights for weighted sampling
    fine_distribution = df_train["fine label"].value_counts().tolist()
    class_weights = [1 / df_train["fine label"].value_counts()[i]
                     for i in range(num_coarse_label, num_all_label)]
    class_weights = [0] * num_coarse_label + class_weights
    image_weights = [class_weights[i] for i in df_train['fine label']]
    weight_sampler = torch.utils.data.WeightedRandomSampler(
        image_weights, len(df_train))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              num_workers=4, pin_memory=True, sampler=weight_sampler)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

    return train_loader, test_loader


class ClearCache:
    def __init__(self, device: torch.device):
        self.device_backend = {'cuda': torch.cuda,
                               'cpu': None}[device]

    def __enter__(self):
        if self.device_backend:
            self.device_backend.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.device_backend:
            self.device_backend.empty_cache()


class FineTuner(torch.nn.Module, ABC):
    def __str__(self) -> str:
        return self.__class__.__name__.split('Fine')[0].lower()

    def __len__(self) -> int:
        return sum(p.numel() for p in self.parameters())

# TODO: Whenever there is the change in loss function, check the implementation accordingly, to whether include
# softmax or sigmoid on classifier or not


class VITFineTuner(FineTuner):
    def __init__(self,
                 vit_model_index: int,
                 num_classes: int):
        super().__init__()
        vit_model_name = ['b_16',
                          'b_32',
                          'l_16',
                          'l_32',
                          'h_14']
        self.vit_model_name = vit_model_name[vit_model_index]
        if self.vit_model_name == 'b_16':
            vit_model = torchvision.models.vit_b_16
        elif self.vit_model_name == 'b_32':
            vit_model = torchvision.models.vit_b_32
        elif self.vit_model_name == 'l_16':
            vit_model = torchvision.models.vit_l_16
        elif self.vit_model_name == 'l_32':
            vit_model = torchvision.models.vit_l_32
        elif self.vit_model_name == 'h_14':
            vit_model = torchvision.models.vit_h_14
        else:
            # Handle the case when the model name is not recognized
            raise ValueError(f"Invalid vit_model_name: {self.vit_model_name}")

        vit_weights = eval(f"torchvision.models.ViT_{'_'.join([s.upper() for s in self.vit_model_name.split('_')])}"
                           f"_Weights.DEFAULT")
        self.vit = vit_model(weights=vit_weights)
        self.vit.heads[-1] = torch.nn.Linear(in_features=self.vit.hidden_dim,
                                             out_features=num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.vit(x)
        return x

    def __str__(self):
        return f'{super().__str__()}_{self.vit_model_name}'

# TODO: Whenever there is the change in loss function, check the implementation accordingly, to whether include
# softmax or sigmoid on classifier or not


class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label d. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class d.
    """

    def __init__(self):
        super(LogitsToPredicate, self).__init__()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, d):
        probs = self.sigmoid(x)
        out = torch.sum(probs * d, dim=1)
        return out


def compute_sat_normally(base_model,
                         logits_to_predicate,
                         data, labels_coarse, labels_fine,
                         coarse_label_dict, fine_label_dict,
                         coarse_to_fine, fine_grain_only=False, train_mode=False):
    """
    compute satagg function for rules
    argument:
      - base_model: get probability of the class
      - logits_to_predicate: get the satisfaction of a variable given the label
      - data, labels_coarse, labels_fine
      - coarse_label_dict, fine_label_dict,
      - coarse_to_fine
      - fine_grain_only: if true, the sat is changed accordingly
      - train: whether to train model again, when data is still not convert to prediction yet

    return:
      sat_agg: sat_agg for all the rules

    """
    Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
    And = ltn.Connective(ltn.fuzzy_ops.AndProd())
    Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
    Forall = ltn.Quantifier(
        ltn.fuzzy_ops.AggregPMeanError(p=4), quantifier="f")
    SatAgg = ltn.fuzzy_ops.SatAgg()

    if train_mode:
        prediction = base_model(data)
    else:
        prediction = data

    x = ltn.Variable("x", prediction)

    x_variables = {}
    for name, label in fine_label_dict.items():
        x_variables[label] = ltn.Variable(
            name, prediction[labels_fine == label])
    for name, label in coarse_label_dict.items():
        x_variables[label] = ltn.Variable(
            name, prediction[labels_coarse == label])

    sat_agg_list = []
    sat_agg_label = []

    # Coarse labels: for all x[i], x[i] -> l[i]

    for i in coarse_label_dict.values():
        if x_variables[i].value.numel() != 0:
            sat_agg_label.append(
                f'for all (coarse label) x[{i}], x[{i}] -> l[{i}]')
            sat_agg_list.append(
                Forall(x_variables[i], logits_to_predicate(x_variables[i], l[i])))

    # TODO: double check the rule
    # Coarse Label: for all x[coarse], - {x[different coarse]}

    # for i in range(len(coarse_label_dict)):
    #     for j in range(len(coarse_label_dict)):
    #         if i != j and x_variables[i].value.numel() != 0:
    #             sat_agg_list.append(
    #                 Forall(x_variables[i], Not(logits_to_predicate(x_variables[i], l[j]))))

    # Coarse Label: for all x[coarse], - {x[coarse] and x[different coarse]}
        for i in coarse_label_dict.values():
            for j in coarse_label_dict.values():
                if i != j :
                    sat_agg_list.append(Forall(x, Not(And(logits_to_predicate(x, l[i]), logits_to_predicate(x, l[j])))))

    # Fine to coarse label: for all x[fine], x[fine] and x[correspond coarse]

    for label_coarse, label_fine_list in coarse_to_fine.items():
        for label_fine in label_fine_list:
            if x_variables[label_fine].value.numel() != 0:
              sat_agg_list.append(Forall(x_variables[label_fine],
                                            And(logits_to_predicate(x_variables[label_fine], l[label_fine]), logits_to_predicate(x_variables[label_fine], l[label_coarse])))
                                    )

    # Fine labels: for all x[i], x[i] -> l[i]

    for i in fine_label_dict.values():
        if x_variables[i].value.numel() != 0:
            sat_agg_list.append(
                Forall(x_variables[i], logits_to_predicate(x_variables[i], l[i])))

    # TODO: Double check the rule
    # Fine Label: for all x[fine], - {x[different fine]}

    # for i in range(len(fine_label_dict)):
    #         for j in range(len(fine_label_dict)):
    #             if i != j and x_variables[i].value.numel() != 0:
    #                 sat_agg_list.append(
    #                     Forall(x_variables[i], Not(logits_to_predicate(x_variables[i], l[j]))))

    # Fine Label: for all x[fine], -{x[fine], x[diff_fine]}

    for _, label_fine_list in coarse_to_fine.items():
        for label_fine in label_fine_list:
            for i in label_fine_list:
                if (x_variables[label_fine].value.numel() != 0) and (i != label_fine):
                    sat_agg_list.append(Forall(x_variables[label_fine],
                                        Not(logits_to_predicate(x_variables[label_fine], l[i]))))

    sat_agg = SatAgg(
        *sat_agg_list
    )
    return sat_agg


def train(dataloader,
          base_model: FineTuner, logits_to_predicate,
          beta,
          epoch,
          optimizer,
          scheduler,
          loss_mode,
          fine_grain_only=False, mode='normal',
          device=torch.device('cpu'),
          coarse_label_dict={}, fine_label_dict={}, coarse_to_fine={}):
    """
    Train the model using the provided dataloader for one epoch.

    Args:
        dataloader (DataLoader): Dataloader for training data.
        base_model (FineTuner): The model to be trained.
        logits_to_predicate: Function to convert logits to predicates.
        beta (float): specify proportion of ltn and normal loss
        epoch (int): training iteration
        fine_grain_only (bool): If True, train only on fine-grained labels.
        mode (str): Training mode: 'normal', 'ltn_normal', or 'ltn_combine'
        coarse_label_dict (dict, optional): Dictionary mapping coarse labels to numerical labels. Default is an empty dictionary.
        fine_label_dict (dict, optional): Dictionary mapping fine labels to numerical labels. Default is an empty dictionary.
        coarse_to_fine (dict, optional): Dictionary mapping coarse labels to corresponding fine labels. Default is an empty dictionary..

    Returns:
        float: Running loss.
        float: Precision for fine-grained labels.
        float: Recall for fine-grained labels.
        float: Precision for coarse labels.
        float: Recall for coarse labels.
    """
    num_coarse_label = len(coarse_label_dict)
    num_fine_label = len(fine_label_dict)
    num_all_label = num_fine_label+num_coarse_label
    loss_fc = nn.CrossEntropyLoss()

    base_model.train()
    size = len(dataloader)
    running_loss = 0.0

    fine_label_ground_truth = []
    fine_label_prediction = []
    coarse_label_ground_truth = []
    coarse_label_prediction = []

    with tqdm(total=size) as pbar:
        description = "Epoch " + str(epoch)
        pbar.set_description_str(description)

        for batch_idx, (data, labels_coarse, labels_fine, image_path) in enumerate(dataloader):
            optimizer.zero_grad(set_to_none=True)

            # Put image to device
            data, labels_coarse, labels_fine = data.to(
                device), labels_coarse.to(device), labels_fine.to(device)
            labels_coarse_one_hot = torch.nn.functional.one_hot(
                labels_coarse, num_classes=num_all_label).float()
            labels_fine_one_hot = torch.nn.functional.one_hot(
                labels_fine, num_classes=num_all_label).float()

            # make prediction
            prediction = base_model(data)

            labels_one_hot = labels_fine_one_hot + labels_coarse_one_hot

            if mode == 'normal':
                if loss_mode == 'binary':
                    loss_fc = torch.nn.BCEWithLogitsLoss()
                    loss = loss_fc(prediction, labels_one_hot)
                elif loss_mode == 'marginal':
                    loss_fc = torch.nn.MultiLabelMarginLoss()
                    loss = loss_fc(prediction, labels_one_hot.long())
                elif loss_mode == 'softmarginal':
                    loss_fc = torch.nn.MultiLabelSoftMarginLoss()
                    loss = loss_fc(prediction, labels_one_hot)

            if mode == 'ltn_normal':
                sat_agg = compute_sat_normally(base_model, logits_to_predicate,
                                               prediction, labels_coarse, labels_fine,
                                               coarse_label_dict, fine_label_dict, coarse_to_fine,
                                               fine_grain_only)
                loss = 1. - sat_agg

            if mode == 'ltn_combine':
                sat_agg = compute_sat_normally(base_model, logits_to_predicate,
                                               prediction, labels_coarse, labels_fine,
                                               coarse_label_dict, fine_label_dict, coarse_to_fine,
                                               fine_grain_only)
                if loss_mode == 'binary':
                    loss_fc = torch.nn.BCEWithLogitsLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot))
                elif loss_mode == 'marginal':
                    loss_fc = torch.nn.MultiLabelMarginLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot.long()))

                elif loss_mode == 'softmarginal':
                    loss_fc = torch.nn.MultiLabelSoftMarginLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot))

            running_loss += loss.item()

            # Backpropagation
            loss.backward()

            torch.nn.utils.clip_grad_norm_(base_model.parameters(), 10.0)
            optimizer.step()
            running_loss += loss

            # Accuracy evaluation of coarse and fine grain
            prediction = prediction.cpu().detach()

            # Get coarse label prediction and ground truth
            prediction_coarse_label = prediction[:, :num_coarse_label]
            coarse_label_prediction_batch = torch.argmax(
                prediction_coarse_label, dim=1)
            coarse_label_prediction.extend(coarse_label_prediction_batch)
            coarse_label_ground_truth.extend(labels_coarse.cpu().detach())

            # Get fine label prediction and ground truth
            prediction_fine_label = prediction[:, num_coarse_label:]
            fine_label_prediction_batch = torch.argmax(
                prediction_fine_label, dim=1)
            fine_label_prediction.extend(fine_label_prediction_batch)
            fine_label_ground_truth.extend(
                labels_fine.cpu().detach() - num_coarse_label)

            pbar.update()

        # Compute running loss
        running_loss = running_loss / size

        # Compute evaluation metrics
        accuracy_fine = accuracy_score(
            fine_label_ground_truth, fine_label_prediction, normalize=True)
        precision_fine = precision_score(
            fine_label_ground_truth, fine_label_prediction, average='macro')
        recall_fine = recall_score(
            fine_label_ground_truth, fine_label_prediction, average='macro')
        accuracy_coarse = accuracy_score(
            coarse_label_ground_truth, coarse_label_prediction, normalize=True)
        precision_coarse = precision_score(
            coarse_label_ground_truth, coarse_label_prediction, average='macro')
        recall_coarse = recall_score(
            coarse_label_ground_truth, coarse_label_prediction, average='macro')

        # print evaluation metric:

        pbar.set_postfix_str(" epoch %d | loss %.4f | Train coarse acc %.3f |Train coarse Prec %.3f | Train coarse Rec %.3f | Train fine acc %.3f |Train fine Prec %.3f | Train fine Rec %.3f" %
                             (epoch, running_loss, accuracy_coarse, precision_coarse, recall_coarse, accuracy_fine, precision_fine, recall_fine))

        # Update learning rate
        scheduler.step()

        save_metric = [float(running_loss.detach().to("cpu")),
                       accuracy_fine, precision_fine, recall_fine,
                       accuracy_coarse, precision_coarse, recall_coarse]

    return save_metric


@torch.no_grad()
def valid(dataloader,
          base_model, logits_to_predicate,
          beta,
          loss_mode,
          fine_grain_only=False, mode='normal',
          device=torch.device('cpu'),
          coarse_label_dict={}, fine_label_dict={}, coarse_to_fine={},):
    """
    Validate the model using the provided dataloader.

    Args:
        dataloader (DataLoader): Dataloader for validation data.
        base_model (FineTuner): The model to be evaluated.
        logits_to_predicate (function): Function to convert logits to predicates.
        beta: specify proportion of ltn and normal loss
        fine_grain_only (bool, optional): If True, validate only on fine-grained labels. Default is False.
        mode (str, optional): Validation mode: 'normal', 'ltn_normal', or 'ltn_combine'. Default is 'normal'.
        device (torch.device, optional): Device to perform computations on. Default is 'cuda'.
        coarse_label_dict (dict, optional): Dictionary mapping coarse labels to numerical labels. Default is an empty dictionary.
        fine_label_dict (dict, optional): Dictionary mapping fine labels to numerical labels. Default is an empty dictionary.
        coarse_to_fine (dict, optional): Dictionary mapping coarse labels to corresponding fine labels. Default is an empty dictionary.
        model_name (string, optional): Name of the model to save, default is empty string

    Returns:
        float: Running loss.
        float: Precision for fine-grained labels.
        float: Recall for fine-grained labels.
        float: Precision for coarse labels.
        float: Recall for coarse labels.
    """
    num_coarse_label = len(coarse_label_dict)
    num_fine_label = len(fine_label_dict)
    num_all_label = num_fine_label + num_coarse_label
    base_model.eval()
    size = len(dataloader)
    running_loss = 0.0
    fine_label_ground_truth = []
    fine_label_prediction = []
    coarse_label_ground_truth = []
    coarse_label_prediction = []

    with tqdm(total=size) as pbar:
        description = "Evaluate test set: "
        pbar.set_description_str(description)

        for batch_idx, (data, labels_coarse, labels_fine, image_path) in enumerate(dataloader):
            # Put image to device
            data, labels_coarse, labels_fine = data.to(
                device), labels_coarse.to(device), labels_fine.to(device)

            # get ground truth
            labels_coarse_one_hot = torch.nn.functional.one_hot(
                labels_coarse, num_classes=num_all_label).float()
            labels_fine_one_hot = torch.nn.functional.one_hot(
                labels_fine, num_classes=num_all_label).float()

            # make prediction
            prediction = base_model(data)

            labels_one_hot = labels_fine_one_hot + labels_coarse_one_hot

            if mode == 'normal':
                if loss_mode == 'binary':
                    loss_fc = torch.nn.BCEWithLogitsLoss()
                    loss = loss_fc(prediction, labels_one_hot)
                elif loss_mode == 'marginal':
                    loss_fc = torch.nn.MultiLabelMarginLoss()
                    loss = loss_fc(prediction, labels_one_hot.long())
                elif loss_mode == 'softmarginal':
                    loss_fc = torch.nn.MultiLabelSoftMarginLoss()
                    loss = loss_fc(prediction, labels_one_hot)

            if mode == 'ltn_normal':
                sat_agg = compute_sat_normally(base_model, logits_to_predicate,
                                               prediction, labels_coarse, labels_fine,
                                               coarse_label_dict, fine_label_dict, coarse_to_fine,
                                               fine_grain_only)
                loss = 1. - sat_agg

            if mode == 'ltn_combine':
                sat_agg = compute_sat_normally(base_model, logits_to_predicate,
                                               prediction, labels_coarse, labels_fine,
                                               coarse_label_dict, fine_label_dict, coarse_to_fine,
                                               fine_grain_only)
                if loss_mode == 'binary':
                    loss_fc = torch.nn.BCEWithLogitsLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot))
                elif loss_mode == 'marginal':
                    loss_fc = torch.nn.MultiLabelMarginLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot.long()))

                elif loss_mode == 'softmarginal':
                    loss_fc = torch.nn.MultiLabelSoftMarginLoss()
                    loss = beta*(1. - sat_agg) + (1 - beta) * \
                        (loss_fc(prediction, labels_one_hot))

            running_loss += loss.item()

            # Accuracy evaluation of coarse and fine grain
            prediction = prediction.cpu().detach()

            prediction_coarse_label = prediction[:, :num_coarse_label]
            coarse_label_prediction_batch = torch.argmax(
                prediction_coarse_label, dim=1)
            coarse_label_prediction.extend(coarse_label_prediction_batch)
            coarse_label_ground_truth.extend(labels_coarse.cpu().detach())

            prediction_fine_label = prediction[:, num_coarse_label:]
            fine_label_prediction_batch = torch.argmax(
                prediction_fine_label, dim=1)
            fine_label_prediction.extend(fine_label_prediction_batch)
            fine_label_ground_truth.extend(
                labels_fine.cpu().detach() - num_coarse_label)

            pbar.update()

        # Compute running loss
        running_loss = running_loss / size

        # Compute evaluation metrics
        accuracy_fine = accuracy_score(
            fine_label_ground_truth, fine_label_prediction, normalize=True)
        precision_fine = precision_score(
            fine_label_ground_truth, fine_label_prediction, average='macro')
        recall_fine = recall_score(
            fine_label_ground_truth, fine_label_prediction, average='macro')
        accuracy_coarse = accuracy_score(
            coarse_label_ground_truth, coarse_label_prediction, normalize=True)
        precision_coarse = precision_score(
            coarse_label_ground_truth, coarse_label_prediction, average='macro')
        recall_coarse = recall_score(
            coarse_label_ground_truth, coarse_label_prediction, average='macro')

        # print the training metrics

        pbar.set_postfix_str(" loss %.4f | Train coarse acc %.3f |Train coarse Prec %.3f | Train coarse Rec %.3f | Train fine acc %.3f |Train fine Prec %.3f | Train fine Rec %.3f" %
                             (running_loss, accuracy_coarse, precision_coarse, recall_coarse, accuracy_fine, precision_fine, recall_fine))

        save_metric = [running_loss,
                       accuracy_fine, precision_fine, recall_fine,
                       accuracy_coarse, precision_coarse, recall_coarse]

    return save_metric


def transform_evaluation_metric(metric_list):
    transformed_metrics = []
    for metric_dict in metric_list:
        try:
            transformed_metrics.append({
                'running_loss': metric_dict[0],
                'accuracy_fine': metric_dict[1],
                'precision_fine': metric_dict[2],
                'recall_fine': metric_dict[3],
                'accuracy_coarse': metric_dict[4],
                'precision_coarse': metric_dict[5],
                'recall_coarse': metric_dict[6]
            })
        except:
            print('error in getting some metric')
    return transformed_metrics


def save_evaluation_metric(evaluation_metric_train_raw, evaluation_metric_valid_raw, path: str, description):
    """
    Save the evaluation metric plot.

    Args:
        path (str): File path to save the plot.
        evaluation_metric_train (list): List of dictionaries containing evaluation metrics for training data.
        evaluation_metric_valid (list): List of dictionaries containing evaluation metrics for validation data.
        description (str)

    Returns:
        None
    """

    num_epochs = len(evaluation_metric_train_raw)
    evaluation_metric_train = transform_evaluation_metric(
        evaluation_metric_train_raw)
    evaluation_metric_valid = transform_evaluation_metric(
        evaluation_metric_valid_raw)
    y_limits = [0.0, 1.0]

    for metric in ['running_loss', 'accuracy_fine', 'precision_fine', 'recall_fine', 'accuracy_coarse', 'precision_coarse', 'recall_coarse']:

        plt.figure(figsize=(10, 6))
        plt.plot(range(num_epochs), [
                 element[metric] for element in evaluation_metric_train], label='Training', color='green')
        plt.plot(range(num_epochs), [
                 element[metric] for element in evaluation_metric_valid], label='Validation', color='blue')
        plt.ylim(y_limits)
        plt.xlabel('Epoch')
        plt.ylabel('Metric Value')
        plt.title(f'{metric.capitalize()} Over Epochs')
        plt.legend()
        plt.grid(True)

        save_path = f'{path}/{description}_{metric.capitalize()}.png'

        plt.savefig(save_path)
        plt.close()  # Close the plot to clear the memory


def calculate_metrics_per_label(y_true: List[int], y_pred: List[int],
                                labels: List[int]) -> Tuple[List[float], List[float], List[float], List[List[int]]]:
    """
    Calculates precision, recall, F1 score, and confusion matrix for each label.

    Args:
        y_true (List[int]): True labels.
        y_pred (List[int]): Predicted labels.
        labels (List[int]): List of label indices.

    Returns:
        Tuple[List[float], List[float], List[float], List[List[int]]]: Precision, recall, F1 score, and confusion matrix.

    """
    # accuracy_per_label = accuracy_score(y_true, y_pred)
    precision_per_label = precision_score(
        y_true, y_pred, average=None, labels=labels)
    recall_per_label = recall_score(
        y_true, y_pred, average=None, labels=labels)
    accuracy_per_label = [precision * recall for precision,
                          recall in zip(precision_per_label, recall_per_label)]
    f1_per_label = f1_score(y_true, y_pred, average=None, labels=labels)
    confusion_mat = confusion_matrix(y_true, y_pred, labels=labels)

    return accuracy_per_label, precision_per_label, recall_per_label, f1_per_label, confusion_mat


def save_metrics_to_excel(y_true: List[int], y_pred: List[int],
                          label_dict: Dict[int, str],
                          model_name: str, path: str, description: str) -> None:
    """
    Calculates metrics per label and saves the results to an Excel file.

    Args:
        y_true (List[int]): True labels.
        y_pred (List[int]): Predicted labels.
        label_dict (Dict[int, str]): Dictionary mapping label indices to label names.
        model_name (str): Name of the model.
        path (str): Directory where the Excel file will be saved.
        description (str)
    """
    label_temp = [i for i in label_dict.values()]
    accuracy, precision, recall, f1, confusion = calculate_metrics_per_label(y_true, y_pred, label_temp)
    
    # Create a list to store dictionaries with metrics
    metrics_list = []
    
    inverse_dict = {value: key for key, value in label_dict.items()}
    
    for label_idx, label in enumerate(label_temp):
        metrics_dict = {
            'Label': inverse_dict[int(label)],
            'Accuracy': accuracy[label_idx],
            'Precision': precision[label_idx],
            'Recall': recall[label_idx],
            'F1': f1[label_idx],
            'True Positives': confusion[label_idx][label_idx],
            'True Negatives': confusion.sum() - confusion[label_idx].sum() - confusion[:, label_idx].sum() + confusion[label_idx][label_idx],
            'False Positives': confusion[:, label_idx].sum() - confusion[label_idx][label_idx],
            'False Negatives': confusion[label_idx].sum() - confusion[label_idx][label_idx]
        }
        metrics_list.append(metrics_dict)
    
    # Create DataFrame from the list of dictionaries
    metrics_df = pd.DataFrame(metrics_list, columns=['Label', 'Accuracy', 'Precision', 'Recall', 'F1', 'True Positives', 'True Negatives', 'False Positives', 'False Negatives'])


    metrics_df.to_excel(
        f'{path}/{description}_coarse_grained_{model_name}_test_metric.xlsx', index=False)


def save_confusion_matrices(num_coarse_label: int, num_all_labels: int,
                            base_model, test_loader: DataLoader,
                            save_path: str,
                            fine_grain_only: bool,
                            description: str,) -> None:
    """
    Compute and save confusion matrices for coarse and fine labels based on the predictions
    from the provided base_model and test_loader. Save the generated matrices as images.

    Args:
        num_coarse_label (int): Number of coarse labels.
        num_all_labels (int): Total number of labels (including coarse and fine labels).
        base_model: PyTorch model for prediction.
        test_loader (DataLoader): DataLoader containing test data.
        save_path (str): Path to save the generated confusion matrix images.
        fine_grain_only (bool): Train fine grain only or not
        description (str)

    Returns:
        None
    """
    coarse_index = slice(num_coarse_label)
    fine_index = slice(num_coarse_label, num_all_labels)

    coarse_label_ground_truth = []
    coarse_label_prediction = []
    fine_label_ground_truth = []
    fine_label_prediction = []
    image_path_list = []

    print("Save confusion matrices")

    # Iterate through the test data and make predictions
    for batch_idx, (data, labels_coarse, labels_fine, image_path) in enumerate(test_loader):
        data = data.to(device)

        prediction = base_model(data).cpu().detach()

        prediction_coarse_label = prediction[:, :num_coarse_label]
        coarse_label_prediction_batch = torch.argmax(
            prediction_coarse_label, dim=1)
        coarse_label_prediction.extend(coarse_label_prediction_batch)
        coarse_label_ground_truth.extend(labels_coarse)

        prediction_fine_label = prediction[:, num_coarse_label:]
        fine_label_prediction_batch = torch.argmax(
            prediction_fine_label, dim=1) + num_coarse_label
        fine_label_prediction.extend(fine_label_prediction_batch)
        fine_label_ground_truth.extend(labels_fine)

        # get image path
        image_path_list.extend(image_path)

    # Compute confusion matrix for coarse labels
    confusion_matrix_coarse = metrics.confusion_matrix(
        coarse_label_ground_truth, coarse_label_prediction)
    display_labels_coarse = [str(label)
                             for label in range(num_coarse_label)]

    # Plot and save coarse label confusion matrix
    fig_coarse, ax_coarse = plt.subplots(figsize=(15, 15))
    cm_display_coarse = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_coarse,
                                               display_labels=display_labels_coarse)
    cm_display_coarse.plot(ax=ax_coarse, values_format='d')
    ax_coarse.set_title('Coarse Label Confusion Matrix')
    plt.savefig(
        f'{save_path}/{description}_coarse_label_confusion_matrix.png')
    plt.close(fig_coarse)

    print('Saved coarse label confusion matrix successfully')

    # Compute confusion matrix for fine labels
    confusion_matrix_fine = metrics.confusion_matrix(
        fine_label_ground_truth, fine_label_prediction)
    display_labels_fine = [str(label) for label in range(
        num_coarse_label, num_all_labels)]

    # Plot and save fine label confusion matrix
    fig_fine, ax_fine = plt.subplots(figsize=(15, 15))
    cm_display_fine = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_fine,
                                             display_labels=display_labels_fine)
    cm_display_fine.plot(ax=ax_fine, values_format='d')
    ax_fine.set_title('Fine Label Confusion Matrix')
    plt.savefig(f'{save_path}/{description}_fine_label_confusion_matrix.png')
    plt.close(fig_fine)

    print('Saved fine label confusion matrix successfully')

    if not fine_grain_only:
        print('save coarse grain excel file')
        save_metrics_to_excel(coarse_label_ground_truth, coarse_label_prediction,
                              coarse_label_dict, base_model,
                              save_path,
                              description + '_coarse'
                              )

    print('save fine grain excel file')
    save_metrics_to_excel(fine_label_ground_truth, fine_label_prediction,
                          fine_label_dict, base_model,
                          save_path,
                          description + '_fine'
                          )

    print('save excel file successfully')

    # Save result for later use

    coarse_label_prediction = np.array(coarse_label_prediction)
    coarse_label_ground_truth = np.array(coarse_label_ground_truth)
    image_path_list = np.array(image_path_list)

    # Concatenate the arrays along the second axis (axis=1)
    concatenated_array = np.column_stack(
        (coarse_label_prediction, coarse_label_ground_truth, image_path_list))

    # Save the concatenated array
    with open(f'{save_path}/concatenated_data_coarse_{description}.pkl', 'wb') as pickle_file:
        pickle.dump(concatenated_array, pickle_file)

    print('Save concatenated coarse data successfully!')

    fine_label_prediction = np.array(fine_label_prediction)
    fine_label_ground_truth = np.array(fine_label_ground_truth)
    image_path_list = np.array(image_path_list)

    # Concatenate the arrays along the second axis (axis=1)
    concatenated_array = np.column_stack(
        (fine_label_prediction, fine_label_ground_truth, image_path_list))

    # Save the concatenated array
    with open(f'{save_path}/concatenated_data_fine_{description}.pkl', 'wb') as pickle_file:
        pickle.dump(concatenated_array, pickle_file)

    print('Save concatenated fine data successfully!')





In [3]:
def hyper_parameter_tune(config):
    # Load dataset
    df_train, df_test = process_image_folders(
        base_train_folder, base_test_folder)
    train_loader, test_loader = create_data_loaders(
        df_train, df_test, image_resize, batch_size, num_coarse_label, num_all_label)

    # Model Initialization
    base_model = VITFineTuner(vit_model_index, num_output).to(device)
    logits_to_predicate = ltn.Predicate(LogitsToPredicate()).to(ltn.device)

    # Training Configuration
    optimizer = torch.optim.Adam(base_model.parameters(), config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, 2, 0.95)
    beta = config['beta']

    evaluation_metric_train = []
    evaluation_metric_valid = []
    accuracy_recent_coarse = []
    accuracy_recent_fine = []

    # Update description:
    description = 'model ' + str(vit_model_index) + \
        ' ' + "ltn_combine" + ' ' + "softmarginal" + \
        ' ' + str(config['lr']) + ' ' + str(config['beta']) + ' ' + str(datetime.now().strftime("%Y-%m-%d"))
    print(description)

    for epoch in range(loaded_epoch, num_epochs):
        with ClearCache(device):
            evaluation_metric_train.append(train(train_loader,
                                                  base_model, logits_to_predicate,
                                                  beta,
                                                  epoch,
                                                  optimizer,
                                                  scheduler,
                                                  loss_mode,
                                                  fine_grain_only, mode,
                                                  device,
                                                  coarse_label_dict, fine_label_dict, coarse_to_fine))
            evaluation_metric_valid.append(valid(test_loader,
                                                  base_model, logits_to_predicate,
                                                  beta,
                                                  loss_mode,
                                                  fine_grain_only, mode,
                                                  device,
                                                  coarse_label_dict, fine_label_dict, coarse_to_fine))

            accuracy_recent_coarse.append(evaluation_metric_valid[-1][1])
            accuracy_recent_fine.append(evaluation_metric_valid[-1][4])

            # Save best checkpoint according to sum accuracy
            if max(accuracy_recent_coarse) == accuracy_recent_coarse[-1] and max(accuracy_recent_fine) == accuracy_recent_fine[-1]:
                torch.save({"model_state_dict": base_model.state_dict(),
                            "optimizer_state_dict": optimizer.state_dict(),
                            "scheduler": scheduler.state_dict()},
                            f"{base_path}/model/model_{description}.pth")
                print(f"Saved PyTorch Model State to {description}")
            
            # Saving evaluation_metric_train
            with open(f'{base_path}/model/evaluation_metric_train_{description}.pkl', 'wb') as f:
                pickle.dump(evaluation_metric_train, f)

            # Saving evaluation_metric_valid
            with open(f'{base_path}/model/evaluation_metric_valid_{description}.pkl', 'wb') as f:
                pickle.dump(evaluation_metric_valid, f)

        print('#' * 100)

    # Create a folder with the name 'description' inside the 'result' folder
    result_folder_path = os.path.join(base_path, "result", description)
    os.makedirs(result_folder_path, exist_ok=True)

    save_evaluation_metric(evaluation_metric_train,
                           evaluation_metric_valid, result_folder_path, description)

    # Save confusion matrices to the result folder with the description
    save_confusion_matrices(num_coarse_label, num_all_label,
                            base_model, test_loader, result_folder_path,
                            fine_grain_only, description)


In [4]:
# Assigning argparse values to variables
base_path = "/home/ngocbach"
mode = "ltn_combine"
vit_model_index = 2
beta = 0.8
lr = 0.00001
fine_grain_only = False
loss_mode = "softmarginal"
load_checkpoint = False
num_epochs = 5
batch_size = 16
description = 'model ' + str(0) + \
    ' ' + "ltn_combine" + ' ' + "softmarginal" + \
    ' ' + str(0.00001) + ' ' + str(0.8) + " " + str(datetime.now().strftime("%Y-%m-%d"))
print(description)


# All label
category_dict = {
    'Air Defence': ['30N6E', 'Iskander', 'Pantsir-S1', 'Rs-24'],
    'BMP': ['BMP-1', 'BMP-2', 'BMP-T15'],
    'BTR': ['BRDM', 'BTR-60', 'BTR-70', 'BTR-80'],
    'Tank': ['T-14', 'T-62', 'T-64', 'T-72', 'T-80', 'T-90'],
    'SPA': ['2S19_MSTA', 'BM-30', 'D-30', 'Tornado', 'TOS-1'],
    'BMD': ['BMD'],
    'MT_LB': ['MT_LB']
}

coarse_label_dict, fine_label_dict, coarse_to_fine = create_label_dict(
    category_dict)

# Print the resulting dictionaries
print("coarse_label_dict:")
print(coarse_label_dict)
print("\nfine_label_dict:")
print(fine_label_dict)
print("\ncoarse_to_fine:")
print(coarse_to_fine)

l = create_one_hot_tensors(fine_label_dict, coarse_label_dict)
inverse_dict = create_inverse_dict(coarse_label_dict, fine_label_dict)

# Constants and Configuration
image_resize = 224
num_coarse_label = len(coarse_label_dict)
num_fine_label = len(fine_label_dict)
num_all_label = num_fine_label + num_coarse_label
num_output = num_all_label
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)
base_train_folder = f"{base_path}/dataset/train"
base_test_folder = f"{base_path}/dataset/test"
load_checkpoint_path = f"{base_path}/model/model_{description}.pth"

# Load dataset
df_train, df_test = process_image_folders(
    base_train_folder, base_test_folder)
train_loader, test_loader = create_data_loaders(
    df_train, df_test, image_resize, batch_size, num_coarse_label, num_all_label)

print('get dataset successfully')

# Model Initialization
base_model = VITFineTuner(vit_model_index, num_output).to(device)
logits_to_predicate = ltn.Predicate(LogitsToPredicate()).to(ltn.device)
print('model initialization successfully')

# Training Configuration
optimizer = torch.optim.Adam(base_model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, 0.1)

if load_checkpoint:
    # Load the checkpoint
    checkpoint = torch.load(load_checkpoint_path)

    # Load model and optimizer states
    base_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    # Load scheduler state, if available in the checkpoint
    if 'scheduler' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler'])

        # Retrieve the epoch information if available in the checkpoint
        loaded_epoch = scheduler.last_epoch

    # Restoring evaluation_metric_train.
    with open(f'{base_path}/model/evaluation_metric_train_{description}.pkl', 'rb') as f:
        evaluation_metric_train = pickle.load(f)

    # Restoring evaluation_metric_valid
    with open(f'{base_path}/model/evaluation_metric_valid_{description}.pkl', 'rb') as f:
        evaluation_metric_valid = pickle.load(f)

    print('load checkpoint successfully')

else:
    print('train from beginning')
    loaded_epoch = 0  # If not loading a checkpoint, start training from epoch 0




# Define the hyperparameter search space
# TODO: change hyperparameter search space
search_space = {
    "lr": tune.loguniform(1e-6, 1e-5),
    "beta": tune.quniform(0.2, 0.8, 0.2),
}

# hyperopt_search = HyperOptSearch(search_space,
#                                   metric="accuracy_recent_coarse",
#                                   mode="max")
search_space = {
    "lr": tune.grid_search([0.000005]),
    "beta": tune.grid_search([0.75, 1]),
}


model 0 ltn_combine softmarginal 1e-05 0.8 2023-12-01
coarse_label_dict:
{'Air Defence': 0, 'BMP': 1, 'BTR': 2, 'Tank': 3, 'SPA': 4, 'BMD': 5, 'MT_LB': 6}

fine_label_dict:
{'30N6E': 7, 'Iskander': 8, 'Pantsir-S1': 9, 'Rs-24': 10, 'BMP-1': 11, 'BMP-2': 12, 'BMP-T15': 13, 'BRDM': 14, 'BTR-60': 15, 'BTR-70': 16, 'BTR-80': 17, 'T-14': 18, 'T-62': 19, 'T-64': 20, 'T-72': 21, 'T-80': 22, 'T-90': 23, '2S19_MSTA': 24, 'BM-30': 25, 'D-30': 26, 'Tornado': 27, 'TOS-1': 28, 'BMD': 29, 'MT_LB': 30}

coarse_to_fine:
{0: [7, 8, 9, 10], 1: [11, 12, 13], 2: [14, 15, 16, 17], 3: [18, 19, 20, 21, 22, 23], 4: [24, 25, 26, 27, 28], 5: [29], 6: [30]}
Using device:  cuda


/home/ngocbach/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


get dataset successfully
model initialization successfully
train from beginning


In [ ]:

# Initialize Ray and start hyperparameter tuning
# Resource will be used accordingly. The default is for gg colab notebook
ray.shutdown()
ray.init(num_cpus=2, num_gpus=1, ignore_reinit_error=True)

# define tuner object
# TODO: Change metric when necessary
results = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(hyper_parameter_tune),
        resources={"cpu": 2, "gpu": 1}
    ),
    tune_config=tune.TuneConfig(
        num_samples=1,
    ),
    param_space = search_space
)
results.fit()


In [7]:
config = {}
config['lr'] = 5e-06
config['beta'] = 1
# Update description:
description = 'model ' + str(vit_model_index) + \
    ' ' + "ltn_combine" + ' ' + "softmarginal" + \
    ' ' + str(config['lr']) + ' ' + str(config['beta']) + ' ' + str("2023-11-29")
print(description)

model 2 ltn_combine softmarginal 5e-06 0.75 2023-11-29


In [16]:
# Load the checkpoint
load_checkpoint_path = "/home/ngocbach/model/model_model 2 ltn_combine softmarginal 5e-06 1 2023-11-30.pth"
checkpoint = torch.load(load_checkpoint_path)

# Load model and optimizer states
base_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# Load scheduler state, if available in the checkpoint
if 'scheduler' in checkpoint:
    scheduler.load_state_dict(checkpoint['scheduler'])

    # Retrieve the epoch information if available in the checkpoint
    loaded_epoch = scheduler.last_epoch

print('load checkpoint successfully')
    
# Create a folder with the name 'description' inside the 'result' folder
result_folder_path = os.path.join(base_path, "result", description)
os.makedirs(result_folder_path, exist_ok=True)

# # Save confusion matrices to the result folder with the description
# save_confusion_matrices(num_coarse_label, num_all_label,
#                         base_model, test_loader, result_folder_path,
#                         fine_grain_only, description)

load checkpoint successfully


In [17]:
coarse_index = slice(num_coarse_label)
fine_index = slice(num_coarse_label, num_all_label)

coarse_label_ground_truth = []
coarse_label_prediction = []
fine_label_ground_truth = []
fine_label_prediction = []
image_path_list = []
# Iterate through the test data and make predictions
for batch_idx, (data, labels_coarse, labels_fine, image_path) in enumerate(test_loader):
    data = data.to(device)

    prediction = base_model(data).cpu().detach()

    prediction_coarse_label = prediction[:, coarse_index]
    coarse_label_prediction_batch = torch.argmax(
        prediction_coarse_label, dim=1)
    coarse_label_prediction.extend(coarse_label_prediction_batch)
    coarse_label_ground_truth.extend(labels_coarse)

    prediction_fine_label = prediction[:, fine_index]
    fine_label_prediction_batch = torch.argmax(
        prediction_fine_label, dim=1) + num_coarse_label
    fine_label_prediction.extend(fine_label_prediction_batch)
    fine_label_ground_truth.extend(labels_fine)

    # get image path
    image_path_list.extend(image_path)

/home/ngocbach/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
# Assuming coarse_label and fine_label are lists of tensors
coarse_label = coarse_label_prediction
fine_label = fine_label_prediction

# Your coarse_to_fine dictionary
coarse_to_fine = {0: [7, 8, 9, 10], 1: [11, 12, 13], 2: [14, 15, 16, 17],
                  3: [18, 19, 20, 21, 22, 23], 4: [24, 25, 26, 27, 28],
                  5: [29], 6: [30]}

# Convert tensors to integers
coarse_label = [int(label.item()) for label in coarse_label]
fine_label = [int(label.item()) for label in fine_label]

count = 0
# Count of pairs without one-to-one correspondence
for i in range(len(coarse_label_prediction)):
    if (fine_label[i] in coarse_to_fine[coarse_label[i]]):
        count = count + 1


In [19]:
print("num consistancy:" ,count / len(coarse_label_prediction))

num consistancy: 0.32510795805058607


In [20]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# Assuming coarse_label, fine_label, coarse_label_ground_truth, and fine_label_ground_truth are lists of tensors
# Your coarse_to_fine dictionary
coarse_to_fine = {0: [7, 8, 9, 10], 1: [11, 12, 13], 2: [14, 15, 16, 17],
                  3: [18, 19, 20, 21, 22, 23], 4: [24, 25, 26, 27, 28],
                  5: [29], 6: [30]}

# Calculate F1 and accuracy for coarse labels
f1_coarse = f1_score(coarse_label_ground_truth, coarse_label_prediction, average='weighted')
accuracy_coarse = accuracy_score(coarse_label_ground_truth, coarse_label_prediction)

# Calculate F1 and accuracy for fine labels
f1_fine = f1_score(fine_label_ground_truth, fine_label_prediction, average='weighted')
accuracy_fine = accuracy_score(fine_label_ground_truth, fine_label_prediction)

print(f"F1 Score for Coarse Labels: {f1_coarse}")
print(f"Accuracy for Coarse Labels: {accuracy_coarse}")
print(f"F1 Score for Fine Labels: {f1_fine}")
print(f"Accuracy for Fine Labels: {accuracy_fine}")


F1 Score for Coarse Labels: 0.7878155753312605
Accuracy for Coarse Labels: 0.7896360271437385
F1 Score for Fine Labels: 0.3334929576165714
Accuracy for Fine Labels: 0.3232572486119679


# Get consistancy

In [19]:
import pickle
import numpy as np

def load_numpy_array_from_pickle(file_path):
    with open(file_path, 'rb') as file:
        # Load the NumPy array from the pickle file
        data = pickle.load(file)
    return data

# Replace with the actual path to your pickle file
file_path_coarse = '/home/ngocbach/result/model 2 ltn_combine softmarginal 5e-06 0 2023-11-28/concatenated_data_coarse_model 2 ltn_combine softmarginal 5e-06 0 2023-11-28.pkl'
file_path_fine = "/home/ngocbach/result/model 2 ltn_combine softmarginal 5e-06 0 2023-11-28/concatenated_data_fine_model 2 ltn_combine softmarginal 5e-06 0 2023-11-28.pkl"
loaded_array_coarse = load_numpy_array_from_pickle(file_path_coarse)
loaded_array_fine = load_numpy_array_from_pickle(file_path_fine)

# modify loaded_array_fine to get original prediction that match the dictionary (for ltn_fine_coarse only)
# loaded_array_fine[:,0] = loaded_array_fine[:,0].astype(np.int32) + 7
# loaded_array_fine[:,1] = loaded_array_fine[:,1].astype(np.int32) + 7

In [21]:
import pandas as pd

def process_arrays(input_coarse, input_fine):
    # First task: Extract the desired part from image_path for input_coarse
    image_paths1 = [path.split('/')[-3:] for path in input_coarse[:, 2]]
    image_paths1 = ['/'.join(path) for path in image_paths1]

    # Second task: Extract the desired part from image_path for input_fine
    image_paths2 = [path.split('/')[-3:] for path in input_fine[:, 2]]
    image_paths2 = ['/'.join(path) for path in image_paths2]

    # Create DataFrames for input_coarse and input_fine
    df = pd.DataFrame({
        'image_path': image_paths1,
        'ground_truth_coarse': input_coarse[:, 0],
        'prediction_coarse': input_coarse[:, 1],
        'ground_truth_fine': input_fine[:, 0],
        'prediction_fine': input_fine[:, 1]
    })

    return df

# Example usage:
# Assuming 'loaded_array1' and 'loaded_array2' are the NumPy arrays with shape [length_dataset, 3]
# Replace 'loaded_array1' and 'loaded_array2' with the actual loaded NumPy arrays in your code
df_concatenated = process_arrays(loaded_array_coarse, loaded_array_fine)

In [22]:
import pandas as pd

# Assuming 'df' is the DataFrame with the structure you provided
# Replace 'df' with the actual DataFrame in your code

# Define the coarse_to_fine dictionary
coarse_to_fine = {0: [7, 8, 9, 10], 1: [11, 12, 13], 2: [14, 15, 16, 17],
                  3: [18, 19, 20, 21, 22, 23], 4: [24, 25, 26, 27, 28],
                  5: [29], 6: [30]}

# Create a function to calculate inconsistency
def calculate_inconsistency(row):
    coarse_prediction = int(row['prediction_coarse'])
    fine_prediction = int(row['prediction_fine'])

    if fine_prediction not in coarse_to_fine[coarse_prediction]:
        return 1
    else:
        return 0

# Apply the function to each row in the DataFrame to calculate inconsistency
df_concatenated['inconsistency'] = df_concatenated.apply(calculate_inconsistency, axis=1)

# Calculate the total inconsistency count
total_inconsistency_count = df_concatenated['inconsistency'].sum()

# Calculate the inconsistency rate
inconsistency_rate = total_inconsistency_count / len(df_concatenated)

print("Number of inconsistencies between coarse grain and fine grain predictions:", total_inconsistency_count)
print("Inconsistency rate:", inconsistency_rate)


Number of inconsistencies between coarse grain and fine grain predictions: 0
Inconsistency rate: 0.0


In [39]:
df_concatenated

,image_path,ground_truth_coarse,prediction_coarse,ground_truth_fine,prediction_fine,inconsistency
0,BMD/BMD/6 4.53.52 PM.jpg,1,5,30,29,0
1,BMD/BMD/82 4.53.53 PM.jpg,3,5,24,29,0
2,BMD/BMD/55 4.53.52 PM.jpg,3,5,29,29,0
3,BMD/BMD/25 4.53.52 PM.jpg,1,5,29,29,0
4,BMD/BMD/57 4.53.52 PM.jpg,3,5,29,29,0
...,...,...,...,...,...,...
1616,BTR/BRDM/35.jpg,4,2,14,14,0
1617,BTR/BRDM/28.jpg,2,2,14,14,0
1618,BTR/BRDM/61.jpg,2,2,14,14,0
1619,BTR/BRDM/51.jpg,3,2,18,14,0
